In [48]:
import os
import pandas as pd

In [49]:
path = '/content/drive/My Drive/Hackathon_2020/data'
results = os.path.join(path, 'results')

In [90]:
df_news = pd.read_csv(os.path.join(results, 'old_news', 'all_news.csv'),
                           header = 0, sep = ',', error_bad_lines=False, encoding = "ISO-8859-1", parse_dates = True)
df_news.columns

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['Unnamed: 0', 'Unnamed: 0.1', 'coordinates', 'created_at', 'hashtags',
       'media', 'urls', 'favorite_count', 'id', 'in_reply_to_screen_name',
       'in_reply_to_status_id', 'in_reply_to_user_id', 'lang', 'place',
       'possibly_sensitive', 'retweet_count', 'retweet_id',
       'retweet_screen_name', 'source', 'text', 'tweet_url', 'user_created_at',
       'user_screen_name', 'user_default_profile_image', 'user_description',
       'user_favourites_count', 'user_followers_count', 'user_friends_count',
       'user_listed_count', 'user_location', 'user_name', 'user_screen_name.1',
       'user_statuses_count', 'user_time_zone', 'user_urls', 'user_verified',
       'is_retweet', 'text_noHT_MT', 'query', 'query_score'],
      dtype='object')

In [51]:
df_news = df_news[['created_at', 'favorite_count', 'id','place',
       'possibly_sensitive', 'retweet_count', 'text',  'user_followers_count', 'user_friends_count',
       'user_listed_count', 'user_location', 'user_name','is_retweet', 'text_noHT_MT', 'query', 'query_score']]

In [9]:
set(df_news['query'].values)

{'5G and COVID-19 were linked',
 '5G radiation weakens the immune system',
 'anti-depressants',
 'anti-vaxxer movement targeting Bill Gates as well as the anti-tech movement',
 'anxiety',
 'cognitive psychology',
 'depression during corona',
 'mental therapy',
 'nightmares',
 'psychologist',
 'sadness',
 'therapist',
 'therapy',
 'things like 5G are responsible for some terrible things going on',
 'uneasy'}

In [91]:
news_topics = {'COVID-19 and the 5G Conspiracy Theory': 'COVID 5G Conspiracy',
               '5G and COVID-19 were linked': 'COVID 5G Conspiracy',
               'things like 5G are responsible for some terrible things going on': 'COVID 5G Conspiracy',
               '5G radiation weakens the immune system': 'COVID 5G Conspiracy',
               'anti-vaxxer movement targeting Bill Gates as well as the anti-tech movement': 'Bill Gates Conspiracy',
               'depression during corona': 'Mental Health', 'sadness': 'Mental Health', 'anxiety': 'Mental Health','nightmares': 'Mental Health',
               'uneasy': 'Mental Health','psychologist': 'Mental Health','therapist': 'Mental Health','therapy': 'Mental Health','cognitive psychology': 'Mental Health',
               'mental therapy':'Mental Health','anti-depressants':'Mental Health',
               'Trump supports wearing masks': 'Trump actions',
               '- Trump supports mask wearing "We are United in our effort to defeat the Invisible China Virus, and many people say that it is Patriotic to wear a face mask when you can’t socially distance. There is nobody more Patriotic than me, your favorite President': 'Trump actions'
               }

In [92]:
df_news['topic'] = df_news['query'].apply(lambda x: news_topics[x])

In [69]:
df_news.to_csv(os.path.join(results, 'all_news_ripon.csv'))

In [93]:
set(df_news['topic'].values)

{'Bill Gates Conspiracy', 'COVID 5G Conspiracy', 'Mental Health'}

In [94]:
#df_news = df_news.drop(['query'], axis = 1)
df_news.drop_duplicates(inplace=True)


In [96]:
from afinn import Afinn
af = Afinn()
def get_sentiment(df_news):
  # compute sentiment scores (polarity) and labels
  sentiment_scores = [af.score(tweet) for tweet in df_news['text_noHT_MT'].values]
  sentiment_category = ['positive' if score > 0 
                            else 'negative' if score < 0 
                                else 'neutral' 
                                    for score in sentiment_scores]
  df_news['sentiment_score'] = sentiment_scores
  df_news['sentiment_category']  = sentiment_category
  return df_news 

In [97]:
%%time
df_news = df_news[df_news['text_noHT_MT'].notna()]
df_news = get_sentiment(df_news)

CPU times: user 1min 5s, sys: 23.9 ms, total: 1min 5s
Wall time: 1min 5s


In [ ]:
df_5g = df_news[df_news['topic'] == 'COVID 5G Conspiracy'].sort_values(['query_score'], ascending= False)
df_5g

In [55]:
!pip install afinn

In [98]:
df_news['is_retweet'] = df_news['text'].apply(lambda x: x[:2]=='RT')
print("Total number of Retweets: {}".format(df_news['is_retweet'].sum()))
df_news = df_news[df_news['is_retweet'] == False]

Total number of Retweets: 17254


In [147]:
df = df_news[df_news['retweet_count'] == 762]
print(df[['text']]['text'].iloc[0])

Roughly 100 LAPD officers protested Saturday in front of their headquarters in downtown Los Angeles, urging the public and leaders to "defend the police." https://t.co/0ihAClMkbV


In [127]:
print(df[['text']]['text'].iloc[0])

How could this happen? https://t.co/L8YXyJPDiI


In [ ]:
A surprising new study found that the controversial antimalarial drug hydroxychloroquine helped Covid-19 patients better survive in the hospital.
https://t.co/j6zs4SI2Su

In [ ]:
df_event_phrases_dates = pd.read_csv(os.path.join(path, 'media_details.csv'))
df_event_phrases_dates

In [ ]:
df_news[['user_name', 'user_screen_name', 'user_screen_name.1']]

In [ ]:
event_dates = dict(zip(df_event_phrases_dates['name'], df_event_phrases_dates.account_tye))
event_dates

In [108]:
df_news['media_category'] = df_news['user_name'].apply(lambda x: get_type(x))

In [107]:
def get_type(x):
  if x in event_dates.keys():
    return event_dates[x]
  else:
    return 

In [109]:
df_news['media_category'].value_counts()

media                 58160
healthcare_experts     9302
President               416
Name: media_category, dtype: int64